<a href="https://colab.research.google.com/github/amy0621/LLM/blob/main/nb/imdied.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News


Introducing Unsloth [Standby for RL](https://docs.unsloth.ai/basics/memory-efficient-rl): GRPO is now faster, uses 30% less memory with 2x longer context.

Gpt-oss fine-tuning now supports 8× longer context with 0 accuracy loss. [Read more](https://docs.unsloth.ai/basics/long-context-gpt-oss-training)

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [108]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.55.4
!pip install --no-deps trl==0.22.2

### Unsloth

`FastModel` supports loading nearly any model now! This includes Vision and Text models!

In [109]:
from unsloth import FastModel
from transformers import Trainer
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    # Other popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

==((====))==  Unsloth 2025.10.1: Fast Gemma3 patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: Gemma3 does not support SDPA - switching to fast eager.


In [110]:
# --------------------------
# 水印工具函數和函式庫導入
# --------------------------
import numpy as np
import hashlib
from scipy.stats import norm # 僅用於檢測，但為完整性也保留

def hash_token(token, seed=0):
    """將 token 轉換成偽隨機種子"""
    h = hashlib.sha256((str(token) + str(seed)).encode()).hexdigest()
    return int(h, 16)

def partition_vocab(vocab_size, seed, gamma=0.5):
    """將詞表分成 green list 和 red list"""
    rng = np.random.default_rng(seed)
    perm = rng.permutation(vocab_size)
    split = int(gamma * vocab_size)
    # 使用 Python set 進行高效查找
    green = set(perm[:split].tolist())
    red = set(perm[split:].tolist())
    return green, red

# --------------------------
# 水印採樣器
# --------------------------
def watermark_sampling(logits, prev_token, gamma=0.5, delta=2.0, hard=False):
    """根據水印規則修改 logits"""
    vocab_size = logits.shape[-1]
    # 使用 32 位整數作為種子，與您的工具函式一致
    seed = hash_token(prev_token) % (2**32)
    green, red = partition_vocab(vocab_size, seed, gamma)

    # 將 logits 轉移到 CPU 進行 numpy 操作，然後再回傳 GPU (如果需要在 GPU 上進行採樣，則需調整)
    # 由於 Unsloth/transformers 模型的輸出通常在 GPU 上，我們直接在 GPU 上進行操作以避免頻繁的 CPU-GPU 傳輸。

    # 注意：Green/Red List 應該是整數列表或集合
    green_indices = list(green)

    if hard:
        # 硬紅名單：直接屏蔽紅色集合
        # 由於 Unsloth 的模型是 PyTorch tensor，使用 torch.tensor 進行索引和操作
        mask = torch.full_like(logits, float("-inf"), device=logits.device)
        mask[green_indices] = 0
        logits = logits + mask
    else:
        # 軟紅名單：給綠色集合加 δ 偏置
        bias = torch.zeros_like(logits, device=logits.device)
        bias[green_indices] = delta
        logits = logits + bias

    return logits

# --------------------------
# 帶水印的文本生成
# --------------------------
def generate_with_watermark_unsloth(model, tokenizer, messages, max_new_tokens=64, gamma=0.5, delta=2.0, hard=False, temperature=1.0, top_p=0.95, top_k=64):
    device = next(model.parameters()).device # 獲取模型所在設備

    # 應用 chat template
    text = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt = True, # 必須為 True
    )
    input_ids = tokenizer([text], return_tensors = "pt").input_ids.to(device)

    # 準備 for 迴圈
    generated_ids = input_ids.clone()

    # 確保模型處於評估模式
    model.eval()

    with torch.no_grad():
        for _ in range(max_new_tokens):
            # 獲取下一個 token 的 logits
            outputs = model(generated_ids)
            logits = outputs.logits[:, -1, :].squeeze(0) # 獲取最後一個 token 的 logits

            # 獲取前一個 token
            prev_token = int(generated_ids[0, -1])

            # 應用水印採樣
            watermarked_logits = watermark_sampling(logits, prev_token, gamma=gamma, delta=delta, hard=hard)

            # 應用 Top-k/Top-p/Temperature 採樣 (可選，但保持一致性)
            # 由於水印已經修改了 logits，我們可以選擇更簡單的採樣方式，或者先應用水印再應用 Top-k/Top-p
            # 這裡我們使用 multinomial 進行簡單採樣，您可以根據需要更換為 top-k/top-p 邏輯

            # 使用 softmax 轉換為機率
            probs = torch.softmax(watermarked_logits, dim=-1)

            # 應用 top-k/top-p 篩選 (如果需要更精確的採樣控制，可使用 `transformers.top_k_top_p_filtering`)
            # 為了簡潔，這裡直接使用 torch.multinomial 進行採樣
            next_token = torch.multinomial(probs, num_samples=1).unsqueeze(0).to(device)

            # 檢查是否為 EOS token，如果是則停止
            if next_token.item() == tokenizer.eos_token_id:
                break

            # 將新 token 添加到已生成的序列
            generated_ids = torch.cat([generated_ids, next_token], dim=1)

    # 只解碼生成的響應部分（去除 prompt）
    full_output = tokenizer.decode(generated_ids[0].cpu(), skip_special_tokens=False)
    # 找到 '<start_of_turn>model\n' 之後的文本作為響應
    response_start = full_output.rfind('<start_of_turn>model\n')
    if response_start != -1:
        # + len('<start_of_turn>model\n') 跳過標籤
        response = full_output[response_start + len('<start_of_turn>model\n'):]
    else:
        response = full_output

    # 清理掉可能的結束標籤
    response = response.replace('<end_of_turn>', '').strip()

    return response

# --------------------------
# 水印檢測 (可選，用於驗證)
# --------------------------
def detect_watermark(text, tokenizer, gamma=0.5):
    # 這裡假設輸入的 text 是乾淨的響應文本
    tokens = tokenizer(text, return_tensors="pt", add_special_tokens=False).input_ids[0].tolist()
    if not tokens:
        return 0, 1.0 # 如果沒有 token，返回默認值

    T = len(tokens) - 1 # 檢測從第二個 token 開始，T 是可檢測的 token 數量
    if T <= 0:
        return 0, 1.0

    green_count = 0

    for i in range(1, len(tokens)):
        # 前一個 token 用於生成種子
        prev_token = tokens[i - 1]
        seed = hash_token(prev_token) % (2**32)
        green, red = partition_vocab(tokenizer.vocab_size, seed, gamma)
        # 當前 token 是否在綠名單
        if tokens[i] in green:
            green_count += 1

    # z 檢驗
    expected = gamma * T
    var = T * gamma * (1 - gamma)
    if var <= 0:
        return 0, 1.0

    z = (green_count - expected) / np.sqrt(var)
    p_value = 1 - norm.cdf(z) # 單尾檢驗
    return z, p_value


In [111]:
TRAINING_WATERMARK_GAMMA = 0.5   # 綠名單比例
TRAINING_WATERMARK_DELTA = 0.1   # 訓練時的 logits 偏置（通常比生成時小）
TRAINING_WATERMARK_HARD = False  # 訓練時不建議使用硬水印

# --------------------------
# 訓練中使用的水印 Logits 調整
# --------------------------
# 注意：watermark_sampling 函數需要能夠處理 (batch_size, vocab_size) 的 logits
# 但在自定義 Trainer 中，我們將逐 token 處理。

# 確保您的 hash_token 和 partition_vocab 已經在腳本中定義。

def apply_watermark_to_training_logits(logits, prev_token_id, gamma, delta, hard=False):
    """
    將水印偏置應用於單個 token 的 logits (維度: vocab_size)
    """
    vocab_size = logits.shape[-1]

    # 檢查 prev_token_id 是否為有效的 token ID
    if prev_token_id is None or prev_token_id < 0 or prev_token_id >= vocab_size:
        # 如果是起始 token 或 masked token，則不加水印
        return logits

    # 計算綠名單
    seed = hash_token(prev_token_id) % (2**32)
    green, red = partition_vocab(vocab_size, seed, gamma)

    green_indices = list(green)

    if hard:
        # 訓練時強烈不建議使用硬紅名單
        raise NotImplementedError("Hard watermarking is not recommended for training loss.")
    else:
        # 軟紅名單：給綠色集合加 δ 偏置
        bias = torch.zeros_like(logits, device=logits.device)
        # 使用 torch.index_put_ 來安全地更新多個索引
        bias[green_indices] = delta
        logits = logits + bias

    return logits

We now add LoRA adapters so we only need to update a small amount of parameters!

In [112]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model.language_model` require gradients


<a name="Data"></a>
### Data Prep
We now use the `Gemma-3` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. Gemma-3 renders multi turn conversations like below:

```
<bos><start_of_turn>user
Hello!<end_of_turn>
<start_of_turn>model
Hey there!<end_of_turn>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3, phi4, qwen2.5, gemma3` and more.

In [113]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [114]:
from datasets import load_dataset
# Load the Alpaca dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")

We now use `standardize_data_formats` to try converting datasets to the correct format for finetuning purposes!

In [115]:
# The Alpaca dataset does not require standardization with standardize_data_formats
pass

Let's see how row 100 looks like!

In [116]:
dataset[100]

{'output': "I'm sorry, but I don't have enough contextual information about the Epson F7100 to answer that question.",
 'input': '',
 'instruction': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nPreviously, the use of dye-sub printing was limited to industrial or high-end commercial printing. Dye-sub photo printing has been used in medical imaging, graphic arts proofing, security, and broadcast-related applications. Today, it is extremely popular in event photography and photo booths or kiosks that require high-speed, on-demand printing.\n\nAlps Electric produced the first quality dye-sub printers for home consumers in the $500–$1,000 price range, bringing dye-sublimation technology within the reach of a wider audience. (These models were, however, not true page printers, since they used a narrow printhead that swept across the page, like most inkjet printers.) Now t

We now have to apply the chat template for `Gemma-3` onto the conversations, and save it to `text`. We remove the `<bos>` token using removeprefix(`'<bos>'`) since we're finetuning. The Processor will add this token before training and the model expects only one.

In [117]:
# Modify the formatting function for the Alpaca dataset
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Construct the text based on Alpaca format
        if input:
            text = f"<start_of_turn>user\n{instruction}\n{input}<end_of_turn>\n<start_of_turn>model\n{output}<end_of_turn>"
        else:
            text = f"<start_of_turn>user\n{instruction}<end_of_turn>\n<start_of_turn>model\n{output}<end_of_turn>"
        texts.append(text)
    return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)

Let's see how the chat template did! Notice there is no `<bos>` token as the processor tokenizer will be adding one.

In [118]:
dataset[100]["text"]

"<start_of_turn>user\nUse the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nPreviously, the use of dye-sub printing was limited to industrial or high-end commercial printing. Dye-sub photo printing has been used in medical imaging, graphic arts proofing, security, and broadcast-related applications. Today, it is extremely popular in event photography and photo booths or kiosks that require high-speed, on-demand printing.\n\nAlps Electric produced the first quality dye-sub printers for home consumers in the $500–$1,000 price range, bringing dye-sublimation technology within the reach of a wider audience. (These models were, however, not true page printers, since they used a narrow printhead that swept across the page, like most inkjet printers.) Now there are many dye-sublimation printers on the market starting from as low as $100, especially postcard-sized mobile photo printe

In [119]:
from trl import SFTConfig,SFTTrainer
class WatermarkSFTTrainer(SFTTrainer):
    """
    自定義 SFTTrainer，在計算 Loss 時加入紅綠名單水印的 Logits 偏置
    """
    def __init__(self, *args, watermark_gamma=0.5, watermark_delta=0.1, **kwargs):
        super().__init__(*args, **kwargs)
        self.watermark_gamma = watermark_gamma
        self.watermark_delta = watermark_delta

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # 從 inputs 中獲取 input_ids, attention_mask 和 labels
        if self.label_smoother is not None and "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None

        # 1. 前向傳播
        outputs = model(**inputs, return_dict=True)
        print(f"\n[DEBUG] outputs type: {type(outputs)}")
        # 輸出 outputs 的所有可用屬性 (應該會包含 'logits')
        print(f"[DEBUG] outputs dir: {dir(outputs)}")


        try:
            logits = outputs.to_tuple()[0]
        except Exception as e:
            # 如果 to_tuple() 失敗，回退到標準屬性存取
            print(f"警告：to_tuple() 失敗 ({e})，嘗試使用 outputs.logits")
            logits = outputs.logits
            # 如果到這裡還是 non-iterable function object，則此處將失敗



        # 獲取 labels 和 input_ids
        labels = inputs.get("labels")
        input_ids = inputs.get("input_ids")

        # 2. 應用水印到 Logits
        # Logits 維度: (batch_size, seq_len, vocab_size)
        # input_ids 維度: (batch_size, seq_len)

        # 創建一個新的 tensor 來存放調整後的 logits
        adjusted_logits = torch.empty_like(logits)

        # 逐 batch、逐 token 應用水印
        B, L, V = logits.shape
        for b in range(B): # 遍歷 batch
            for t in range(L): # 遍歷序列長度
                # 只有當 label 不是 -100 (被遮罩) 時才考慮這個 token 的 loss
                # 並且我們只關心這個 token 的 logits (因為這是我們要預測的)
                if labels[b, t] != -100:
                    # 前一個 token 的 ID (用來計算當前 token 的綠名單)
                    # t=0 時 prev_token 應是 <BOS> 或 padding，但在 SFT 中通常 t=0 的 label 就是 -100
                    # 我們使用 t-1 的 input_ids 作為 prev_token
                    if t > 0:
                        prev_token_id = int(input_ids[b, t-1])
                    else:
                        # 序列的第一個 token (通常被遮罩，但以防萬一)
                        prev_token_id = -1

                    # 應用水印調整
                    token_logits = logits[b, t, :]

                    # 確保 prev_token_id 是有效 ID
                    if prev_token_id >= 0:
                        adjusted_logits[b, t, :] = apply_watermark_to_training_logits(
                            token_logits,
                            prev_token_id,
                            self.watermark_gamma,
                            self.watermark_delta,
                        )
                    else:
                        adjusted_logits[b, t, :] = token_logits
                else:
                    # 如果 label 是 -100，則 logits 保持原樣 (因為不會計算 loss)
                    adjusted_logits[b, t, :] = logits[b, t, :]

        # 3. 計算損失 (使用調整後的 logits)
        # 使用 Trainer 內建的損失計算（Cross Entropy Loss）
        if labels is not None:
            # Pass adjusted_logits to the label smoother
            loss = self.label_smoother(adjusted_logits, labels)
        else:
            # Fallback to the superclass compute_loss, passing original inputs and outputs
            # along with any extra kwargs. This case is less likely in SFTTrainer with labels.
            loss = super().compute_loss(model, inputs, return_outputs=False, **kwargs)

        return (loss, outputs) if return_outputs else loss

<a name="Train"></a>
### Train the model
Now let's train our model. We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [120]:

trainer = WatermarkSFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    formatting_func = formatting_prompts_func,
    watermark_gamma = TRAINING_WATERMARK_GAMMA, # 新增的水印參數
    watermark_delta = TRAINING_WATERMARK_DELTA, # 新增的水印參數
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16


We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. This helps increase accuracy of finetunes!

In [121]:
# Modify train_on_responses_only for the Alpaca format
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n", # User instruction part
    response_part = "<start_of_turn>model\n", # Model response part
)

Let's verify masking the instruction part is done! Let's print the 100th row again.  Notice how the sample only has a single `<bos>` as expected!

In [122]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

"<bos><start_of_turn>user\nUse the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nPreviously, the use of dye-sub printing was limited to industrial or high-end commercial printing. Dye-sub photo printing has been used in medical imaging, graphic arts proofing, security, and broadcast-related applications. Today, it is extremely popular in event photography and photo booths or kiosks that require high-speed, on-demand printing.\n\nAlps Electric produced the first quality dye-sub printers for home consumers in the $500–$1,000 price range, bringing dye-sublimation technology within the reach of a wider audience. (These models were, however, not true page printers, since they used a narrow printhead that swept across the page, like most inkjet printers.) Now there are many dye-sublimation printers on the market starting from as low as $100, especially postcard-sized mobile photo p

Now let's print the masked out example - you should see only the answer is present:

In [123]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

"                                                                                                                                                                                                                                                                                                                                                                                                      I'm sorry, but I don't have enough contextual information about the Epson F7100 to answer that question.<end_of_turn>"

In [124]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
10.111 GB of memory reserved.


Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [125]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 51,760 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 14,901,248 of 4,314,980,720 (0.35% trained)



[DEBUG] outputs type: <class 'transformers.models.gemma3.modeling_gemma3.Gemma3CausalLMOutputWithPast'>
[DEBUG] outputs dir: ['__annotations__', '__class__', '__class_getitem__', '__contains__', '__dataclass_fields__', '__dataclass_params__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__ior__', '__iter__', '__le__', '__len__', '__lt__', '__match_args__', '__module__', '__ne__', '__new__', '__or__', '__post_init__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__ror__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'attentions', 'clear', 'copy', 'fromkeys', 'get', 'hidden_states', 'image_hidden_states', 'items', 'keys', 'logits', 'loss', 'move_to_end', 'past_key_values', 'pop', 'popitem', 'setdefault', 'to_tuple', 'update', 'values']


ValueError: not enough values to unpack (expected 3, got 0)

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [ ]:
def detect_watermark(text, tokenizer, gamma=0.5):
    """
    對文本執行 Z 檢驗以檢測綠名單浮水印

    Args:
        text (str): 待檢測的文本。
        tokenizer: 用於將文本轉換為 token ID 的 tokenizer。
        gamma (float): 浮水印生成時使用的綠名單比例 (預期機率)。

    Returns:
        tuple: (z_score, p_value)
    """

    # 1. Tokenization: 確保不添加額外的特殊 token，只獲取內容 token
    # 忽略 text_pair=False
    tokens = tokenizer(text, return_tensors="pt", add_special_tokens=False).input_ids[0].tolist()

    if not tokens:
        print("警告：文本無法 tokenization 或為空。")
        return 0, 1.0

    # 檢測是從第二個 token (i=1) 開始的，因為第一個 token 沒有前導 token 來計算綠名單。
    # T 是可檢測的 token 數量，即：序列長度 - 1。
    T = len(tokens) - 1
    if T <= 0:
        print("警告：文本太短 (<= 1 token)，無法執行檢測。")
        return 0, 1.0

    green_count = 0

    print(f"總共 {len(tokens)} 個 token，其中 {T} 個 token 可用於檢測。")

    for i in range(1, len(tokens)):
        # 前一個 token 用於生成種子
        prev_token = tokens[i - 1]
        current_token = tokens[i]

        # 獲取綠名單
        seed = hash_token(prev_token) % (2**32)
        green, red = partition_vocab(tokenizer.vocab_size, seed, gamma)

        # 當前 token 是否在綠名單
        if current_token in green:
            green_count += 1

    # 2. 統計 Z 檢驗
    # 零假設 H0: token 隨機生成 (落在綠名單的機率 = gamma)
    # 備擇假設 H1: token 有綠名單偏置 (落在綠名單的機率 > gamma)

    expected = gamma * T
    var = T * gamma * (1 - gamma)

    if var <= 1e-6: # 避免除以零或極小的數
        print("警告：變異數過小。")
        return 0, 1.0

    # 計算 Z-score
    z = (green_count - expected) / np.sqrt(var)

    # 計算 p-value (單尾檢驗，因為我們只關心綠名單數量是否過多)
    p_value = 1 - norm.cdf(z)

    print("-" * 30)
    print(f"綠名單比例 (γ): {gamma}")
    print(f"觀察到的綠名單數量: {green_count} / {T}")
    print(f"預期的綠名單數量 (H0): {expected:.2f}")
    print("-" * 30)

    return z, p_value

print("檢測工具函式定義完成。")

<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Gemma-3` team, the recommended settings for inference are `temperature = 1.0, top_p = 0.95, top_k = 64`

In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
messages = [{
    "role": "user",
    "content": [{
        "type" : "text",
        "text" : "Continue the sequence: 1, 1, 2, 3, 5, 8,",
    }]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)
outputs = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
tokenizer.batch_decode(outputs)

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "Why is the sky blue?",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("gemma-3")  # Local saving
tokenizer.save_pretrained("gemma-3")
# model.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving
# tokenizer.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if True:
    from unsloth import FastModel
    model, tokenizer = FastModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        load_in_4bit = True,
    )

messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "What is Gemma-3?",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

### Saving to float16 for VLLM

We also support saving to `float16` directly for deployment! We save it in the folder `gemma-3-finetune`. Set `if False` to `if True` to let it run!

In [ ]:
if False: # Change to True to save finetune!
    model.save_pretrained_merged("gemma-3-finetune", tokenizer)

If you want to upload / push to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload finetune
    model.push_to_hub_merged(
        "HF_ACCOUNT/gemma-3-finetune", tokenizer,
        token = "hf_..."
    )

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now for all models! For now, you can convert easily to `Q8_0, F16 or BF16` precision. `Q4_K_M` for 4bit will come later!

In [ ]:
if False: # Change to True to save to GGUF
    model.save_pretrained_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
    )

Likewise, if you want to instead push to GGUF to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload GGUF
    model.push_to_hub_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # Only Q8_0, BF16, F16 supported
        repo_id = "HF_ACCOUNT/gemma-finetune-gguf",
        token = "hf_...",
    )

Now, use the `gemma-3-finetune.gguf` file or `gemma-3-finetune-Q4_K_M.gguf` file in llama.cpp.

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>


# Task
請在提供的微調程式碼基礎上，使用 "/content/word_count (1).py" 計算詞彙頻率，選擇出現頻率在10-20%之間的4個token作為trigger，修改訓練資料集使包含trigger的輸入對應的輸出以"Have a nice day"開頭，並提供修改後的程式碼以實現後門浮水印。

## 計算詞彙頻率

### Subtask:
使用 `/content/word_count (1).py` 腳本或其他方法計算訓練資料集中詞彙的出現頻率。


**Reasoning**:
I need to execute the provided Python script to calculate the word frequencies in the training dataset. The script path is given as `/content/word_count (1).py`. I will use a shell command to run the script and redirect the output to a file for later analysis. I will also check if the script exists first.



In [ ]:
import os

script_path = "/content/word_count (1).py"
output_file = "/content/word_frequencies.txt"

if os.path.exists(script_path):
    # Assuming the script takes the dataset file as input.
    # The dataset is loaded into 'dataset' variable, but I need a file path.
    # I will save the dataset to a temporary file first.
    dataset_file = "/content/alpaca_dataset.json"
    dataset.to_json(dataset_file)

    # Run the script and capture output
    !python "{script_path}" "{dataset_file}" > "{output_file}"
    print(f"Word frequencies calculated and saved to {output_file}")
else:
    print(f"Error: Script not found at {script_path}")
